In [8]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import time
import os
import glob
import torch
from tqdm import tqdm
from helpers import *
from sklearn.preprocessing import MinMaxScaler


In [9]:
data = pd.read_csv('data_from_news.csv')
data

,Date,State,Occurence
0,2019-01-04,illinois,1
1,2019-01-08,illinois,1
2,2019-01-05,illinois,1
3,2019-01-08,illinois,1
4,2019-01-08,alaska,1
...,...,...,...
9621,2021-04-08,illinois,1
9622,2021-04-07,illinois,1
9623,2021-04-08,illinois,1
9624,2021-04-09,illinois,1


In [13]:
# Group data by weeks
matrix, date = convert_data_to_matrix(data, group_data = True, group_days = 7)
matrix.shape

(119, 50)

In [22]:
scaler = MinMaxScaler(feature_range=(0, 1))

# Normalize data by row
# Transpose input data as MinMaxScaler normalize by columns in default
matrix = scaler.fit_transform(matrix.T).T

In [23]:
x, y = get_x_y(matrix, lookback = 4)
print(x.shape)
print(y.shape)
date = date[:x.shape[0]]
print(date.shape)

(115, 4, 50)
(115, 50)
(115,)


In [24]:
train_loader, test_loader = split_data(x, y)

In [25]:
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(GRU, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn) = self.gru(x, (h0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out